In [1]:
import os, sys
root_path = os.path.abspath('.').split('jupyters')[0]
sys.path.append(root_path)

In [2]:
import json
import argparse
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt

from functools import partial

In [3]:
map_size = (5, 5)
#py_func = partial(data_process, map_size = map_size)

In [4]:
correlations = np.zeros(map_size, np.float32)

In [5]:
correlations

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]], dtype=float32)

In [6]:
def make_corr_map(mp, map_size):
    #mp = mp.numpy()
    correlations = np.zeros(map_size, np.float32)
    for i in range(map_size[0]):
        for j in range(map_size[1]):
            if i+mp == j:
                correlations[i,j] += 1
    return correlations

In [7]:
correlations = make_corr_map(0, (5,5))
print(correlations)

[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]


In [8]:
correlations = make_corr_map(1, (5,5))
print(correlations)

[[0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0.]]


In [9]:
correlations = make_corr_map(2, (5,5))
print(correlations)

[[0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


In [10]:
correlations = make_corr_map(-1, (5,5))
print(correlations)

[[0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]]


In [11]:
correlations = make_corr_map(-2, (5,5))
print(correlations)

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]]


In [12]:
def gen_integer():
    while True:
        yield tf.random.uniform(shape=[1], minval = -3, maxval=4, dtype=tf.int32, seed=10)

In [13]:
def data_process(mp, map_size):
    mp = mp.numpy()
    correlations = np.zeros(map_size, np.float32)
    for i in range(map_size[0]):
        for j in range(map_size[1]):
            if i+mp == j:
                correlations[i,j] += 1
    return mp, correlations

In [14]:
py_func = partial(data_process, map_size = map_size)
def map_function(motion_paramters):
    return tf.py_function(py_func, [motion_paramters], [tf.float32, tf.float32])

In [15]:
ds = tf.data.Dataset.from_generator(gen_integer, output_types=tf.int32)
ds = ds.map(map_function)
ds = ds.batch(100)

In [16]:
for mp, corr in ds.take(1):
    print(mp)
    print(min(mp), max(mp))
    print(corr)

tf.Tensor(
[[ 2.]
 [-3.]
 [ 1.]
 [-3.]
 [ 0.]
 [-1.]
 [ 2.]
 [ 2.]
 [ 3.]
 [-1.]
 [-2.]
 [-1.]
 [ 0.]
 [ 3.]
 [ 0.]
 [ 3.]
 [ 0.]
 [-1.]
 [ 2.]
 [ 0.]
 [ 2.]
 [ 3.]
 [ 2.]
 [ 1.]
 [ 1.]
 [ 3.]
 [ 0.]
 [ 0.]
 [ 3.]
 [ 1.]
 [ 1.]
 [-2.]
 [-2.]
 [ 0.]
 [ 1.]
 [ 1.]
 [-3.]
 [ 2.]
 [ 2.]
 [-1.]
 [-3.]
 [ 0.]
 [ 1.]
 [-2.]
 [-3.]
 [-3.]
 [ 0.]
 [ 3.]
 [-1.]
 [ 3.]
 [ 2.]
 [-3.]
 [-1.]
 [-1.]
 [-1.]
 [ 3.]
 [ 1.]
 [ 2.]
 [ 2.]
 [-3.]
 [-1.]
 [-2.]
 [ 3.]
 [ 2.]
 [ 2.]
 [-1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 3.]
 [ 0.]
 [-2.]
 [-1.]
 [-3.]
 [-1.]
 [ 2.]
 [ 0.]
 [ 0.]
 [ 2.]
 [ 1.]
 [-2.]
 [ 0.]
 [-2.]
 [-3.]
 [ 2.]
 [ 1.]
 [ 2.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 2.]
 [ 2.]
 [-1.]
 [-2.]
 [-1.]
 [ 2.]
 [ 1.]
 [ 3.]
 [ 2.]
 [ 3.]], shape=(100, 1), dtype=float32)
tf.Tensor([-3.], shape=(1,), dtype=float32) tf.Tensor([3.], shape=(1,), dtype=float32)
tf.Tensor(
[[[0. 0. 1. 0. 0.]
  [0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [1

In [17]:
layers = tf.keras.layers
#dir(layers)

In [18]:
regressor = tf.keras.Sequential([layers.Conv1D(3, 3),
                             layers.Conv1D(1, 3),
                             layers.Dense(1)])

In [19]:
pred = regressor(np.ones([16,5,5]))
pred = tf.reshape(pred, [-1, 1])
print(pred)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

tf.Tensor(
[[3.0086262]
 [3.0086262]
 [3.0086262]
 [3.0086262]
 [3.0086262]
 [3.0086262]
 [3.0086262]
 [3.0086262]
 [3.0086262]
 [3.0086262]
 [3.0086262]
 [3.0086262]
 [3.0086262]
 [3.0086262]
 [3.0086262]
 [3.0086262]], shape=(16, 1), dtype=float32)


In [20]:
regressor.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              multiple                  48        
_________________________________________________________________
conv1d_1 (Conv1D)            multiple                  10        
_________________________________________________________________
dense (Dense)                multiple                  2         
Total params: 60
Trainable params: 60
Non-trainable params: 0
_________________________________________________________________


In [21]:
def loss_fn(pred, label):
    loss = tf.sqrt(tf.pow(pred - label, 2))
    loss_mean = tf.reduce_mean(loss)
    return loss_mean

In [22]:
i = 0
x = []
x_loss = []
optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
for epoch in range(2000):
    i+= 1
    x.append(i)
    for mp, correlations in ds.take(1):
        with tf.GradientTape() as tape:
            pred = regressor(correlations)
            pred = tf.reshape(pred, [-1, 1])
            loss = loss_fn(pred, mp)
            x_loss.append(loss)
        gradients = tape.gradient(loss, regressor.trainable_variables)
        optimizer.apply_gradients(zip(gradients, regressor.trainable_variables)) 
        if i % 10 == 0:
            print(i, "iter loss : ", loss.numpy())

10 iter loss :  1.6490821
20 iter loss :  1.6282789
30 iter loss :  1.4059764
40 iter loss :  1.4682002
50 iter loss :  1.3607767
60 iter loss :  1.3837875
70 iter loss :  1.2364552
80 iter loss :  1.3803214
90 iter loss :  1.2270337
100 iter loss :  1.1180118
110 iter loss :  1.1233838
120 iter loss :  0.8941784
130 iter loss :  0.9344401
140 iter loss :  0.9494345
150 iter loss :  0.6427027
160 iter loss :  0.6643688
170 iter loss :  0.6914579
180 iter loss :  0.5946195
190 iter loss :  0.5097621
200 iter loss :  0.57111204
210 iter loss :  0.55393857
220 iter loss :  0.4083116
230 iter loss :  0.3914561
240 iter loss :  0.5536093
250 iter loss :  0.36904407
260 iter loss :  0.40700957
270 iter loss :  0.40685055
280 iter loss :  0.38415188
290 iter loss :  0.19547887
300 iter loss :  0.33504242
310 iter loss :  0.18891697
320 iter loss :  0.24212442
330 iter loss :  0.24854547
340 iter loss :  0.18404011
350 iter loss :  0.16429117
360 iter loss :  0.109488174
370 iter loss :  0.137

In [23]:
correlations = make_corr_map(3, (5,5))
correlations = correlations[np.newaxis, :]
print(correlations)

[[[0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]]


In [24]:
for mp in range(-3, 4):
    print("mp : {}".format(mp))
    correlations = make_corr_map(mp, (5,5))
    print("correlation map : {}".format(correlations))
    correlations = correlations[np.newaxis, :]
    print("pred : {}".format(regressor(correlations)))

mp : -3
correlation map : [[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]]
pred : [[[-2.9948997]]]
mp : -2
correlation map : [[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]]
pred : [[[-1.986789]]]
mp : -1
correlation map : [[0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]]
pred : [[[-0.9848636]]]
mp : 0
correlation map : [[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]
pred : [[[0.05496991]]]
mp : 1
correlation map : [[0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0.]]
pred : [[[1.0475084]]]
mp : 2
correlation map : [[0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
pred : [[[2.0361981]]]
mp : 3
correlation map : [[0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
pred : [[[3.0452375]]]


# learned filter parameters of regressor.

In [29]:
print(regressor.layers[0].get_weights())

[array([[[-0.21616024, -0.07164529,  0.21611726],
        [ 0.37597245,  0.2384891 ,  0.22775249],
        [ 0.10117756,  0.28839284,  0.44226915],
        [-0.62806594, -0.23552941,  0.2362742 ],
        [-0.45106035, -0.01229509,  0.4347627 ]],

       [[-0.14611712, -0.3896284 , -0.66714174],
        [-0.20161796,  0.28394693, -0.03582788],
        [-0.39788336,  0.06856236,  0.07409044],
        [-0.38351083,  0.4414069 , -0.14925236],
        [-0.63133943,  0.3627522 ,  0.7835112 ]],

       [[ 0.54708415, -0.4560891 ,  0.32869297],
        [-0.21942271, -0.013087  ,  0.19574928],
        [-0.07999551, -0.3606549 , -0.18044701],
        [-0.38389963,  0.04623661, -0.12682946],
        [-0.6397276 ,  0.07513396,  0.1364916 ]]], dtype=float32), array([ 0.17347491, -0.03592874, -0.15595496], dtype=float32)]


In [30]:
print(regressor.layers[1].get_weights())

[array([[[-0.9272377 ],
        [-0.18364951],
        [ 0.7815141 ]],

       [[-0.53639185],
        [ 0.63799554],
        [-0.08922239]],

       [[ 0.49577883],
        [ 0.0183778 ],
        [ 0.3135811 ]]], dtype=float32), array([-0.18399233], dtype=float32)]


In [31]:
print(regressor.layers[2].get_weights())

[array([[1.9567003]], dtype=float32), array([-0.10689998], dtype=float32)]


In [32]:
print(regressor.get_weights()[4])

[[1.9567003]]


In [33]:
print(regressor.get_weights()[5])

[-0.10689998]
